In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env


In [29]:
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [30]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)

In [31]:
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [37]:
new_df=pd.DataFrame(df.status_code.value_counts(dropna=False).rename('Count'))

In [40]:
new_df['prob']=df.status_code.value_counts(dropna=False)/len(df)

# Probabilistic Methods 1 Variables(Discrete)

In [41]:
new_df

,Count,prob
200,13958,0.998855
499,16,0.001145


# Probabilistic Methods 2 Variables(Discrete)

In [49]:
newer_df=pd.DataFrame(df[['ip', 'size']].value_counts().rename('Count'))

In [50]:
newer_df['prob']= df[['ip', 'size']].value_counts()/len(df)

In [51]:
newer_df

Count      prob
ip            size                   
97.105.19.58  494819    694  0.049664
              3561      408  0.029197
              493905    184  0.013167
              3659      179  0.012810
              1328      159  0.011378
...                     ...       ...
72.181.105.81 511099      1  0.000072
              511094      1  0.000072
              511076      1  0.000072
              510862      1  0.000072
              512674      1  0.000072

[745 rows x 2 columns]